In [113]:
import pandas as pd

In [215]:
a_group = ['a', 29951, 51, 31, 363, 254, 225, 191]
b_group = ['b', 30089, 58, 35, 413, 279, 230, 171]
c_group = ['c', 30116, 50, 24, 468, 301, 178, 138]

plans = {'Gold': 15.95, 'Silver': 9.95, 'Bronze': 4.95}

In [12]:
ab_test_results = pd.DataFrame([a_group, b_group, c_group], columns=['Version', 'Visitors', 'Gold purchases', 'Gold renewals', 'Silver purchases', 'Silver renewals', 'Bronze purchases', 'Bronze renewals'])

## Data Exploration

In [56]:
ab_test_results

,Version,Visitors,Gold purchases,Gold renewals,Silver purchases,Silver renewals,Bronze purchases,Bronze renewals
0,a,29951,51,31,363,254,225,191
1,b,30089,58,35,413,279,230,171
2,c,30116,50,24,468,301,178,138


In [245]:
def get_conversion_rate_new(version, plan):
    total_purchases = ab_test_results[plan+' purchases'].loc[ab_test_results['Version'] == version].values[0]
    visitors = ab_test_results['Visitors'].loc[ab_test_results['Version'] == version].values[0]
    return round(total_purchases/visitors * 100,2)

def get_conversion_rate_new_total_group(version):
    total_purchases = ab_test_results['Gold purchases'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Silver purchases'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Bronze purchases'].loc[ab_test_results['Version'] == version].values[0]
    visitors = ab_test_results['Visitors'].loc[ab_test_results['Version'] == version].values[0]
    return round(total_purchases/visitors * 100,2)

def get_conversion_rate_renew(version, plan):
    total_renews = ab_test_results[plan+' renewals'].loc[ab_test_results['Version'] == version].values[0]
    previous_amount = ab_test_results[plan+' purchases'].loc[ab_test_results['Version'] == version].values[0]
    return round(total_renews/previous_amount * 100,2)


def get_conversion_rate_renew_total_group(version):
    renewals = ab_test_results['Gold renewals'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Silver renewals'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Bronze renewals'].loc[ab_test_results['Version'] == version].values[0]
    previous_amount = ab_test_results['Gold purchases'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Silver purchases'].loc[ab_test_results['Version'] == version].values[0] + ab_test_results['Bronze purchases'].loc[ab_test_results['Version'] == version].values[0]
    visitors = ab_test_results['Visitors'].loc[ab_test_results['Version'] == version].values[0]
    return round(renewals/previous_amount * 100,2)

In [246]:
get_conversion_rate_renew_total_group('a')

74.49

In [244]:
conversion_rates = pd.DataFrame(data=[
    [get_conversion_rate_new('a', 'Gold'),get_conversion_rate_new('b', 'Gold'), get_conversion_rate_new('c', 'Gold')], [get_conversion_rate_renew('a', 'Gold'),get_conversion_rate_renew('b', 'Gold'), get_conversion_rate_renew('c', 'Gold')], [get_conversion_rate_new('a', 'Silver'),get_conversion_rate_new('b', 'Silver'), get_conversion_rate_new('c', 'Silver')], [get_conversion_rate_renew('a', 'Silver'),get_conversion_rate_renew('b', 'Silver'), get_conversion_rate_renew('c', 'Silver')],[get_conversion_rate_new('a', 'Bronze'),get_conversion_rate_new('b', 'Bronze'), get_conversion_rate_new('c', 'Bronze')], [get_conversion_rate_renew('a', 'Bronze'),get_conversion_rate_renew('b', 'Bronze'), get_conversion_rate_renew('c', 'Bronze')], [get_conversion_rate_new_total_group('a'), get_conversion_rate_new_total_group('b'), get_conversion_rate_new_total_group('c')], [get_conversion_rate_renew_total_group('a'), get_conversion_rate_renew_total_group('b'), get_conversion_rate_renew_total_group('c')]
], columns=['a', 'b', 'c'], index=['Gold new', 'Gold renew', 'Silver new', 'Silver renew', 'Bronze new', 'Bronze renew', 'Total new conversion', 'Total renewal conversion'])
conversion_rates

,a,b,c
Gold new,0.17,0.19,0.17
Gold renew,60.78,60.34,48.00
Silver new,1.21,1.37,1.55
Silver renew,69.97,67.55,64.32
Bronze new,0.75,0.76,0.59
Bronze renew,84.89,74.35,77.53
Total new conversion,2.13,2.33,2.31
Total renewal conversion,74.49,69.19,66.52


In [233]:
get_conversion_rate_renew('c', 'Gold')

48.0

In [36]:
a_conversion_rate_new = get_conversion_rate(ab_test_results, 'a')
b_conversion_rate_new = get_conversion_rate(ab_test_results, 'b')
c_conversion_rate_new = get_conversion_rate(ab_test_results, 'c')

In [53]:
print('conversion rate purchases for A group: '+str(a_conversion_rate_new)+' %')
print('conversion rate purchases for B group: '+str(b_conversion_rate_new)+' %')
print('conversion rate purchases for C group: '+str(c_conversion_rate_new)+' %')

conversion rate purchases for A group: 2.13 %
conversion rate purchases for B group: 2.33 %
conversion rate purchases for C group: 2.31 %


In [213]:
print('total change in conversion of B group: '+str(round(b_conversion_rate_new/a_conversion_rate_new*100-100, 2)) + " %")
print('total change in conversion of C group: '+str(round(c_conversion_rate_new/a_conversion_rate_new*100-100, 2)) + " %")

total change in conversion of B group: 9.39 %
total change in conversion of C group: 8.45 %


## _______
## Analysing the A/B test results

In [130]:
ab_test_results

,Version,Visitors,Gold purchases,Gold renewals,Silver purchases,Silver renewals,Bronze purchases,Bronze renewals
0,a,29951,51,31,363,254,225,191
1,b,30089,58,35,413,279,230,171
2,c,30116,50,24,468,301,178,138


##### If test groups have equal number of visitors?

In [149]:
total_users = ab_test_results['Visitors'].sum()
proportion = pd.DataFrame([round(ab_test_results['Visitors'].loc[ab_test_results['Version'] == 'a'].values[0]/total_users*100, 2), round(ab_test_results['Visitors'].loc[ab_test_results['Version'] == 'b'].values[0]/total_users*100, 2), round(ab_test_results['Visitors'].loc[ab_test_results['Version'] == 'c'].values[0]/total_users*100, 2)], columns=['Percentage of users in each group'], index=['a', 'b', 'c'])
proportion

,Percentage of users in each group
a,33.22
b,33.37
c,33.40


In [151]:
# According to the table, the groups are equally populated.

## Is the result statistically significant?

In [214]:
# We use the assumption we made at the  beginning of the analysis, that results were checked for statistical significance (p-value, standard deriviation, student's t-test)

## Calculate total revenue

In [89]:
def calculate_revenue(plan, version):
    revenue_purchases = round(ab_test_results[plan+' purchases'].loc[ab_test_results['Version'] == version].values[0] * plans.get(plan), 2)
    revenue_renews = round(ab_test_results[plan+' renewals'].loc[ab_test_results['Version'] == version].values[0] * plans.get(plan), 2)
    total_revenue = round(revenue_purchases+revenue_renews, 2)
    return total_revenue, revenue_purchases, revenue_renews

In [90]:
a_gold_revenue = calculate_revenue('Gold', 'a')[0]
a_silver_revenue = calculate_revenue('Silver', 'a')[0]
a_bronze_revenue = calculate_revenue('Bronze', 'a')[0]

In [91]:
b_gold_revenue = calculate_revenue('Gold', 'b')[0]
b_silver_revenue = calculate_revenue('Silver', 'b')[0]
b_bronze_revenue = calculate_revenue('Bronze', 'b')[0]

In [92]:
c_gold_revenue = calculate_revenue('Gold', 'c')[0]
c_silver_revenue = calculate_revenue('Silver', 'c')[0]
c_bronze_revenue = calculate_revenue('Bronze', 'c')[0]

In [216]:
control_group_revenue = a_gold_revenue + a_silver_revenue + a_bronze_revenue
b_group_revenue = b_gold_revenue + b_silver_revenue + b_bronze_revenue
c_group_revenue = c_gold_revenue + c_silver_revenue + c_bronze_revenue

In [237]:
total_revenues = pd.DataFrame([[a_gold_revenue, a_silver_revenue, a_bronze_revenue, control_group_revenue, 0],[b_gold_revenue, b_silver_revenue, b_bronze_revenue, b_group_revenue, round(b_group_revenue/control_group_revenue*100-100, 2)],[c_gold_revenue, c_silver_revenue, c_bronze_revenue, c_group_revenue, round(c_group_revenue/control_group_revenue*100-100, 2)]], columns=['Gold', 'Silver', 'Bronze', 'Total, $', 'Change, %'], index=['Control group', 'B', 'C'])
total_revenues

,Gold,Silver,Bronze,"Total, $","Change, %"
Control group,1307.90,6139.15,2059.20,9506.25,0.00
B,1483.35,6885.40,1984.95,10353.70,8.91
C,1180.30,7651.55,1564.20,10396.05,9.36


In [388]:
total_revenues.to_csv('total_revenue.csv')

In [222]:
## calculate renewals

a_gold_revenue_renew = calculate_revenue('Gold', 'a')[2]
a_silver_revenue_renew = calculate_revenue('Silver', 'a')[2]
a_bronze_revenue_renew = calculate_revenue('Bronze', 'a')[2]

b_gold_revenue_renew = calculate_revenue('Gold', 'b')[2]
b_silver_revenue_renew = calculate_revenue('Silver', 'b')[2]
b_bronze_revenue_renew = calculate_revenue('Bronze', 'b')[2]

c_gold_revenue_renew = calculate_revenue('Gold', 'c')[2]
c_silver_revenue_renew = calculate_revenue('Silver', 'c')[2]
c_bronze_revenue_renew = calculate_revenue('Bronze', 'c')[2]

control_group_revenue_renew = a_gold_revenue_renew + a_silver_revenue_renew + a_bronze_revenue_renew
b_group_revenue_renew = b_gold_revenue_renew + b_silver_revenue_renew + b_bronze_revenue_renew
c_group_revenue_renew = c_gold_revenue_renew + c_silver_revenue_renew + c_bronze_revenue_renew

In [389]:
total_revenues_renew = pd.DataFrame([[a_gold_revenue_renew, a_silver_revenue_renew, a_bronze_revenue_renew, control_group_revenue_renew, 0],[b_gold_revenue_renew, b_silver_revenue_renew, b_bronze_revenue_renew, b_group_revenue_renew, round(b_group_revenue_renew/control_group_revenue_renew*100-100, 2)],[c_gold_revenue_renew, c_silver_revenue_renew, c_bronze_revenue_renew, c_group_revenue_renew, round(c_group_revenue_renew/control_group_revenue_renew*100-100, 2)]], columns=['Gold', 'Silver', 'Bronze', 'Total, $', 'Change, %'], index=['Control group', 'B', 'C'])
total_revenues_renew.to_csv('total_renewals.csv')

total_revenues_renew

,Gold,Silver,Bronze,"Total, $","Change, %"
Control group,494.45,2527.30,945.45,3967.20,0.00
B,558.25,2776.05,846.45,4180.75,5.38
C,382.80,2994.95,683.10,4060.85,2.36


In [390]:
conversion_rates.to_csv('convers_rate.csv')
conversion_rates

,a,b,c
Gold new,0.17,0.19,0.17
Gold renew,60.78,60.34,48.00
Silver new,1.21,1.37,1.55
Silver renew,69.97,67.55,64.32
Bronze new,0.75,0.76,0.59
Bronze renew,84.89,74.35,77.53
Total new conversion,2.13,2.33,2.31
Total renewal conversion,74.49,69.19,66.52


## ___________
## 3. Retention

In [437]:
retention_percent = [91.80,
76.10,
66.00,
57.20,
52.10,
48.70,
45.60,
42.00,
39.60,
37.20,
34.80,
33.40,
32.40,
31.20,
30.00,
28.80,
27.60,
26.40,
25.20,
24.00,
22.80,
21.60,
20.40,
19.20,]
retention_rate = []

for percent in retention_percent:
    rate = round(percent/100, 4)
    retention_rate.append(rate)

In [438]:
base_monthly_signups = 5000
base_conver_rate = 0.02
price_subscr = 10

all_cohorts_revenue = {}
total_sum = 0

for cohort in range(1, 25):
    ## check events in April 2021
    # check April to descrease sigups level
    if cohort == 16:
        base_monthly_signups = base_monthly_signups * 0.85
        base_conver_rate = base_conver_rate * 1.05
    elif cohort > 16:
        base_conver_rate = base_conver_rate * 1.05

    current_signups = base_monthly_signups
    current_conver_rate = base_conver_rate

    #calculating data for cohort
    cohort_revenue = {}

    new_purchases = round(current_signups * current_conver_rate * price_subscr, 2)

    new_purchases_dict = {}
    new_purchases_dict[cohort] = new_purchases
    cohort_revenue.update(new_purchases_dict)
    total_sum = total_sum + new_purchases

    retention_counter = 0
    # calculate renewals for this cohort
    for renewal_month in range(cohort+1,25):
        ## calculate for each cohort
        retentioned_signups = current_signups * retention_rate[retention_counter]
        monthly_renewals = round(retentioned_signups * current_conver_rate * price_subscr, 2)
        total_sum = total_sum + monthly_renewals
        current_month_renewals = {}
        current_month_renewals[renewal_month] = monthly_renewals
        cohort_revenue.update(current_month_renewals)
        retention_counter = retention_counter + 1

    all_cohorts_revenue[cohort] = {}
    all_cohorts_revenue[cohort].update(cohort_revenue)
    #print(cohort)
    #print(current_signups)
    #print(current_conver_rate)
    #print('______')

In [439]:
all_cohorts_revenue

{1: {1: 1000.0,
  2: 918.0,
  3: 761.0,
  4: 660.0,
  5: 572.0,
  6: 521.0,
  7: 487.0,
  8: 456.0,
  9: 420.0,
  10: 396.0,
  11: 372.0,
  12: 348.0,
  13: 334.0,
  14: 324.0,
  15: 312.0,
  16: 300.0,
  17: 288.0,
  18: 276.0,
  19: 264.0,
  20: 252.0,
  21: 240.0,
  22: 228.0,
  23: 216.0,
  24: 204.0},
 2: {2: 1000.0,
  3: 918.0,
  4: 761.0,
  5: 660.0,
  6: 572.0,
  7: 521.0,
  8: 487.0,
  9: 456.0,
  10: 420.0,
  11: 396.0,
  12: 372.0,
  13: 348.0,
  14: 334.0,
  15: 324.0,
  16: 312.0,
  17: 300.0,
  18: 288.0,
  19: 276.0,
  20: 264.0,
  21: 252.0,
  22: 240.0,
  23: 228.0,
  24: 216.0},
 3: {3: 1000.0,
  4: 918.0,
  5: 761.0,
  6: 660.0,
  7: 572.0,
  8: 521.0,
  9: 487.0,
  10: 456.0,
  11: 420.0,
  12: 396.0,
  13: 372.0,
  14: 348.0,
  15: 334.0,
  16: 324.0,
  17: 312.0,
  18: 300.0,
  19: 288.0,
  20: 276.0,
  21: 264.0,
  22: 252.0,
  23: 240.0,
  24: 228.0},
 4: {4: 1000.0,
  5: 918.0,
  6: 761.0,
  7: 660.0,
  8: 572.0,
  9: 521.0,
  10: 487.0,
  11: 456.0,
  12: 420.

In [440]:
list_of_data = []
for cohort in all_cohorts_revenue:
    list_of_data.append(all_cohorts_revenue[cohort])

retention_df = pd.DataFrame(list_of_data, index=list(all_cohorts_revenue.keys()))
retention_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,420.0,396.0,...,312.0,300.0,288.00,276.00,264.00,252.00,240.00,228.00,216.00,204.00
2,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,420.0,...,324.0,312.0,300.00,288.00,276.00,264.00,252.00,240.00,228.00,216.00
3,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,...,334.0,324.0,312.00,300.00,288.00,276.00,264.00,252.00,240.00,228.00
4,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,...,348.0,334.0,324.00,312.00,300.00,288.00,276.00,264.00,252.00,240.00
5,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,...,372.0,348.0,334.00,324.00,312.00,300.00,288.00,276.00,264.00,252.00
6,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,...,396.0,372.0,348.00,334.00,324.00,312.00,300.00,288.00,276.00,264.00
7,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,...,420.0,396.0,372.00,348.00,334.00,324.00,312.00,300.00,288.00,276.00
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,...,456.0,420.0,396.00,372.00,348.00,334.00,324.00,312.00,300.00,288.00
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,...,487.0,456.0,420.00,396.00,372.00,348.00,334.00,324.00,312.00,300.00
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,...,521.0,487.0,456.00,420.00,396.00,372.00,348.00,334.00,324.00,312.00


In [441]:
retention_df.loc['Total for month',:]= retention_df.sum(axis=0)
retention_df.loc[:,'Total for cohort'] = retention_df.sum(axis=1)


In [448]:
retention_df


,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,Total for cohort
1,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,420.0,396.0,...,300.0,288.00,276.00,264.00,252.00,240.00,228.00,216.00,204.00,10149.00
2,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,420.0,...,312.0,300.00,288.00,276.00,264.00,252.00,240.00,228.00,216.00,9945.00
3,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,456.0,...,324.0,312.00,300.00,288.00,276.00,264.00,252.00,240.00,228.00,9729.00
4,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,487.0,...,334.0,324.00,312.00,300.00,288.00,276.00,264.00,252.00,240.00,9501.00
5,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,521.0,...,348.0,334.00,324.00,312.00,300.00,288.00,276.00,264.00,252.00,9261.00
6,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,572.0,...,372.0,348.00,334.00,324.00,312.00,300.00,288.00,276.00,264.00,9009.00
7,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,660.0,...,396.0,372.00,348.00,334.00,324.00,312.00,300.00,288.00,276.00,8745.00
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,761.0,...,420.0,396.00,372.00,348.00,334.00,324.00,312.00,300.00,288.00,8469.00
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,918.0,...,456.0,420.00,396.00,372.00,348.00,334.00,324.00,312.00,300.00,8181.00
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,...,487.0,456.00,420.00,396.00,372.00,348.00,334.00,324.00,312.00,7881.00


In [443]:
retention_df.to_csv('retention.csv')